## Start compute simliarity between two strings

In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
from src.dataset.coupert import CoupertDataset
from src.arguments import DataArguments
from utils.retrieval import retrieval
from utils.metrics import compute_recall
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from safetensors.torch import save_file, load_file

model_dir = "../model/bge_base_en_v1.5"
data_dir = "../data/coupert"
device = "cuda:1"


In [3]:
embs = load_file("../embeddings/embs_bge_base_en.safetensors")
eval_embs = embs["eval_embs"]
gallery_embs = embs["gallery_embs"]

In [4]:
eval_df = pd.read_json("../data/coupert/coupert_eval.jsonl", lines=True)
gallery_df = pd.read_json("../data/coupert/coupert_gallery.jsonl", lines=True)


In [ ]:
gallery_ids = gallery_df["dp-id"].values.tolist()
len(gallery_ids), gallery_embs.shape

In [ ]:
ret_ids = retrieval(
    eval_embs, gallery_embs, gallery_ids=gallery_ids, topK=50, device=device
)
eval_df["bge_base_en_v1.5"] = ret_ids

: 

In [ ]:
ret_ids

In [ ]:
eval_df["bge_recall"] = eval_df.apply(compute_recall("bge_base_en_v1.5"), axis=1)
eval_df.bge_recall.mean()

In [ ]:
eval_df.bge_recall.tolist()

In [ ]:
# 找出recall<0.5的, 查看label的title和预测的title
df = eval_df[eval_df.bge_recall == 0]
for i, row in df.iterrows():
    print("=" * 80)
    print(f"id:{row['dp-id']}")
    print(row["title"])
    print(row["bge_base_en_v1.5"])
    print(row["bge_recall"])
    print(row["label"])
    label = gallery_df[gallery_df["dp-id"].isin(row["label"])]["title"].values
    pred = gallery_df[gallery_df["dp-id"].isin(row["bge_base_en_v1.5"])]["title"].values
    print(label)
    print(pred)

    if i > 10:
        break
    
    

    [
        "3803227877-0",
        "1861470251-0",
        "2128190263-0",
        "1245497872-0",
        "3919540852-0",
        "1707017143-0",
        "2800183383-0",
        "3248669584-0",
        "1643035571-0",
        "1240240909-0",
        "1220245221-0",
    ]